#Importing

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O

In [5]:

import os
print(os.listdir("/content"))
import pandas as pd
from io import StringIO
import io
import string
import random
import json
import pickle
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from scipy.stats import norm

# Any results you write to the current directory are saved as output.

['.config', 'test (1).csv', 'Destinations (1).csv', 'train (1).csv', '.ipynb_checkpoints', 'Destinations.xlsx', 'train.xlsx', 'test.xlsx', 'sample_data']


In [8]:
pip install --user boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 10.2 MB 47.8 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 140 kB 48.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [29]:
# Load train data
train = pd.read_csv('/content/train (1).csv', nrows=100000)

# Load test data
test = pd.read_csv('/content/test (1).csv', nrows=100000)


In [28]:
train.head()

,id,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_mobile,is_package,channel,...,srch_children_cnt,srch_rm_cnt,hotel_continent,hotel_country,hotel_market,stay_dur,no_of_days_bet_booking,Cin_day,Cin_month,Cin_year
0,98854,2,3,66,314,53314,1382.53670,0,0,9,...,0,1,4,8,126,1.0,28.0,7,5,2015
1,98855,2,3,66,314,44951,1587.43911,0,0,9,...,0,1,4,8,110,1.0,26.0,7,5,2015
2,98856,13,1,46,172,3641,1587.43911,0,0,9,...,0,1,6,144,1323,3.0,12.0,27,4,2015
3,98857,13,1,46,172,3641,1587.43911,0,0,10,...,0,1,6,144,1345,1.0,16.0,4,7,2015
4,98858,13,1,46,172,3641,1587.43911,0,0,10,...,0,1,6,144,1013,1.0,5.0,28,9,2015


In [11]:
train.columns

Index(['id', 'date_time', 'site_name', 'posa_continent',
       'user_location_country', 'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'hotel_continent', 'hotel_country', 'hotel_market'],
      dtype='object')

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         1000 non-null   int64  
 1   date_time                  1000 non-null   object 
 2   site_name                  1000 non-null   int64  
 3   posa_continent             1000 non-null   int64  
 4   user_location_country      1000 non-null   int64  
 5   user_location_region       1000 non-null   int64  
 6   user_location_city         1000 non-null   int64  
 7   orig_destination_distance  689 non-null    float64
 8   user_id                    1000 non-null   int64  
 9   is_mobile                  1000 non-null   int64  
 10  is_package                 1000 non-null   int64  
 11  channel                    1000 non-null   int64  
 12  srch_ci                    1000 non-null   object 
 13  srch_co                    1000 non-null   object

#Preprocessing

To clean and pre-process the data and perform exploratory analysis to get some interesting insights into the process of choosing a hotel.



1. Remove the users who did not booked the hotel


2. Identify the searches by each user belonging to a specific type of destination


3. orig_destination_distance contains Nan values

4. The check-in and check-out dates to find the duration of the stay for each of the entries in the training set.

In [14]:
# Check the percentage of Nan in dataset
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train['hotel_country'].count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
orig_destination_distance,311,0.311
id,0,0.000
srch_ci,0,0.000
hotel_country,0,0.000
hotel_continent,0,0.000
srch_destination_type_id,0,0.000
srch_destination_id,0,0.000
srch_rm_cnt,0,0.000
srch_children_cnt,0,0.000
srch_adults_cnt,0,0.000


Cleaning the Data

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         1000 non-null   int64  
 1   date_time                  1000 non-null   object 
 2   site_name                  1000 non-null   int64  
 3   posa_continent             1000 non-null   int64  
 4   user_location_country      1000 non-null   int64  
 5   user_location_region       1000 non-null   int64  
 6   user_location_city         1000 non-null   int64  
 7   orig_destination_distance  689 non-null    float64
 8   user_id                    1000 non-null   int64  
 9   is_mobile                  1000 non-null   int64  
 10  is_package                 1000 non-null   int64  
 11  channel                    1000 non-null   int64  
 12  srch_ci                    1000 non-null   object 
 13  srch_co                    1000 non-null   object

**Adding Extra Features**<br>
Extract relevant information from date columns

**Additional attributes**
<ul>
  <li>stay_dur: number of duration of stay
  <li>no_of_days_bet_booking: number of days between the booking and
  <li>Cin_day: Check-in day
  <li>Cin_month: Check-in month
  <li>Cin_year: Check-out year

In [16]:
# Function to convert date object into relevant attributes
def convert_date_into_days(df):
    df['srch_ci'] = pd.to_datetime(df['srch_ci'])
    df['srch_co'] = pd.to_datetime(df['srch_co'])
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    df['stay_dur'] = (df['srch_co'] - df['srch_ci']).astype('timedelta64[D]')
    df['no_of_days_bet_booking'] = (df['srch_ci'] - df['date_time']).astype('timedelta64[D]')
    
    # For hotel check-in
    # Month, Year, Day
    df['Cin_day'] = df["srch_ci"].apply(lambda x: x.day)
    df['Cin_month'] = df["srch_ci"].apply(lambda x: x.month)
    df['Cin_year'] = df["srch_ci"].apply(lambda x: x.year)

In [17]:
convert_date_into_days(train)

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         1000 non-null   int64         
 1   date_time                  1000 non-null   datetime64[ns]
 2   site_name                  1000 non-null   int64         
 3   posa_continent             1000 non-null   int64         
 4   user_location_country      1000 non-null   int64         
 5   user_location_region       1000 non-null   int64         
 6   user_location_city         1000 non-null   int64         
 7   orig_destination_distance  689 non-null    float64       
 8   user_id                    1000 non-null   int64         
 9   is_mobile                  1000 non-null   int64         
 10  is_package                 1000 non-null   int64         
 11  channel                    1000 non-null   int64         
 12  srch_ci

In [24]:
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train['hotel_country'].count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
id,0,0.0
srch_adults_cnt,0,0.0
Cin_month,0,0.0
Cin_day,0,0.0
no_of_days_bet_booking,0,0.0
stay_dur,0,0.0
hotel_market,0,0.0
hotel_country,0,0.0
hotel_continent,0,0.0
srch_destination_type_id,0,0.0


Fill nan with the day which has max occurence

In [22]:
train['Cin_day'] = train['Cin_day'].fillna(26.0)
train['Cin_month'] = train['Cin_month'].fillna(8.0)
train['Cin_year'] = train['Cin_year'].fillna(2014.0)
train['stay_dur'] = train['stay_dur'].fillna(1.0)
train['no_of_days_bet_booking'] = train['no_of_days_bet_booking'].fillna(0.0)

Fill average values in place for nan, fill with mean

In [23]:

train['orig_destination_distance'].fillna(train['orig_destination_distance'].mean(), inplace=True)

In [25]:
train.head()

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,stay_dur,no_of_days_bet_booking,Cin_day,Cin_month,Cin_year
0,98854,2015-04-08 14:44:00,2,3,66,314,53314,1382.53670,328999,0,...,12439,5,4,8,126,1.0,28.0,7,5,2015
1,98855,2015-04-10 22:06:00,2,3,66,314,44951,1587.43911,328999,0,...,8791,1,4,8,110,1.0,26.0,7,5,2015
2,98856,2015-04-14 08:17:00,13,1,46,172,3641,1587.43911,329012,0,...,11496,3,6,144,1323,3.0,12.0,27,4,2015
3,98857,2015-06-17 14:56:00,13,1,46,172,3641,1587.43911,329012,0,...,28605,6,6,144,1345,1.0,16.0,4,7,2015
4,98858,2015-09-22 14:02:00,13,1,46,172,3641,1587.43911,329012,0,...,21595,6,6,144,1013,1.0,5.0,28,9,2015


Removing datetime object from the dataset

In [26]:
user_id = train['user_id']
columns = ['date_time', 'srch_ci', 'srch_co','user_id','srch_destination_type_id','srch_destination_id']
train.drop(columns=columns,axis=1,inplace=True)

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         1000 non-null   int64  
 1   site_name                  1000 non-null   int64  
 2   posa_continent             1000 non-null   int64  
 3   user_location_country      1000 non-null   int64  
 4   user_location_region       1000 non-null   int64  
 5   user_location_city         1000 non-null   int64  
 6   orig_destination_distance  1000 non-null   float64
 7   is_mobile                  1000 non-null   int64  
 8   is_package                 1000 non-null   int64  
 9   channel                    1000 non-null   int64  
 10  srch_adults_cnt            1000 non-null   int64  
 11  srch_children_cnt          1000 non-null   int64  
 12  srch_rm_cnt                1000 non-null   int64  
 13  hotel_continent            1000 non-null   int64 